In [4]:
import numpy as np
from scipy.special import logit,expit
import collections
import matplotlib.pyplot as plt

In [5]:
def g(x,y):
    y=np.dot(x,y)
    return y

def f(x,y):
    y=np.dot(x,y)
    return y

def pi(x,y):
    y=expit(g(x,y))
    return y

def phi(x,y):
    y=expit(f(x,y))
    return y

def I0(y):
    if y==0:
        return 1

In [6]:
#E-step
def Lih(X,Y,beta):
    Ga=np.zeros((N,2))

    for i in range(N):#响应度
        if Y[i]==1:
            Ga[i,0]=0
            Ga[i,1]=1
        elif Y[i]==0:
            a=(expit(np.dot(X[i],beta[0])))
            b=(1-a)*(1-expit(np.dot(X[i],beta[1])))
            Ga[i,0]=a/(a+b)
            Ga[i,1]=1-Ga[i,0]

    return Ga


#M-step
def Q(X,Y,beta):
    Ga=Lih(X,Y,beta)
    A=np.zeros(N)
    for i in range(N):
        pi=expit(np.dot(beta[0],X[i]))
        phi=expit(np.dot(beta[1],X[i]))
        if Y[i]==0:
            A[i]=(Ga[i,0])*np.log(pi)+(Ga[i,1])*np.log((1-pi)*(1-phi))
            #A[i]=Ga[i,0]*np.log(pi)+Ga[i,1]*np.log((1-pi)*(1-phi))
            #A[i]=(Ga[i,0]*np.log(pi))+(Ga[i,1]*(np.log(1-pi)+np.log(1-phi)))
        else:
            A[i]=(Ga[i,1])*np.log((1-pi)*(phi))
            #A[i]=Ga[i,0]*np.log(pi)+Ga[i,1]*np.log((1-pi)*(1-phi))
            #A[i]=(Ga[i,0]*np.log(pi))+(Ga[i,1]*(np.log(1-pi)+np.log(phi)))
        #A[i]=(Ga[i,0]*(1-Y[i])*np.log(pi))+(Ga[i,1]*(np.log(1-pi)+np.log(1-phi)+Y[i]*np.dot(X[i],beta[1])))
    Q=np.sum(A)
    return Q

def grad(X,Y,Ga,beta):
    A=np.zeros((N,P))
    B=np.zeros((N,P))
    for i in range(N):
        A[i]=((Ga[i,0]*(1-Y[i]))*(1-expit(np.dot(beta[0],X[i])))-(Ga[i,1]*expit(np.dot(beta[0],X[i]))))*X[i]
        B[i]=(Ga[i,1]*(Y[i]-expit(np.dot(beta[1],X[i]))))*X[i]
    grad1=np.sum(A,axis=0)
    grad2=np.sum(B,axis=0)
    grad=np.array([grad1,grad2])
    return grad


#EM
def dgra(X,Y,ibeta,n=1000,t=0.001,xim=0.9,yim=0.3):#梯度下降（回溯梯度下降法）xim为步长缩小倍数

    for i in range(n):#M-step
        lik1=Q(X,Y,ibeta)
        ibeta1=ibeta
        L=Lih(X,Y,ibeta)#E-step
        ibeta=ibeta+t*grad(X,Y,L,ibeta)
        lik=Q(X,Y,ibeta)
        d=np.r_[ibeta[0],ibeta[1]]
        c=np.dot(d,d)
        b=(lik-lik1)
        print('i=',i,'b=',b,'\tstep=',t)
        print('Q=',Q(X,Y,ibeta))
        if b>=yim*t*c :
            t=xim*t
        else:
            break
    return ibeta


In [7]:
N=500
M=50
P=3
np.random.seed(123)


In [8]:
def creatB():
    b1=0.1*np.random.rand(3)-0.05
    b2=np.random.rand(3)-0.5
    b1[2]=-0.5*np.random.rand()-0.5
    b2[1]=0.5*np.random.rand()+0.5
    b=np.array([b1,b2])
    return b

def creatX():#X[i,j]表示第i个样本第j维分量
    X=np.zeros((N,3))
    for i in range(N):
        a=np.zeros(3)
        mu=[-1.5,-1.5]
        cov=np.array([[1,0.3],[0.3,1]])
        a[0]=1
        a[1:]=np.random.multivariate_normal(mu,cov,1,check_valid='raise')
        X[i]=a
    return X


def creatY(X,beta):
    Y=np.zeros(N)
    for i in range(N):
        a=np.random.rand()
        b=pi(X[i],beta[0])
        if a>=b:
            c=np.random.rand()
            d=phi(X[i],beta[1])
            if c>=d:
                Y[i]=1
    return Y

In [6]:
Y=np.zeros(N)
beta=creatB()
A=creatX()
B=expit(np.dot(A,beta.T))
C=np.random.rand(N,2)
for j in range(N):
    if C[j,0]>B[j,0]:
        if C[j,1]<B[j,1]:
            Y[j]=1

In [7]:
np.array(np.where(Y[:]==1)).shape

(1, 29)

In [8]:
X=np.zeros((M,N,3))
Y=np.zeros((M,N))
Betat=np.zeros((M,2,3))
Corr=np.zeros(M)
for i in range(M):
    beta=creatB()
    Betat[i]=beta
    A=creatX()
    X[i]=A
    B=expit(np.dot(A,beta.T))
    C=np.random.rand(N,2)
    for j in range(N):
        if C[j,0]>B[j,0]:
            if C[j,1]<B[j,1]:
                Y[i,j]=1
    Corr[i]=np.array(np.where(Y[i]==1)).shape[1]/N

In [9]:
np.mean(Betat,axis=0)

array([[-0.00372433,  0.0013206 , -0.77501074],
       [-0.02278422,  0.75427055,  0.03106094]])

In [10]:
#initial value
beta0=np.array(np.mean(Betat,axis=0))

In [11]:
hbeta=np.zeros((M,2,3))
for i in range(M):
    hbeta[i]=dgra(X[i],Y[i],beta0)


i= 0 b= 4.8911581815400496 	step= 0.001
Q= -305.0259800673356
i= 1 b= 3.744851909678573 	step= 0.0009000000000000001
Q= -301.281128157657
i= 2 b= 2.944465910535712 	step= 0.0008100000000000001
Q= -298.3366622471213
i= 3 b= 2.3629047209731198 	step= 0.000729
Q= -295.9737575261482
i= 4 b= 1.9271931468173875 	step= 0.0006561000000000001
Q= -294.0465643793308
i= 5 b= 1.5926887210098357 	step= 0.00059049
Q= -292.45387565832095
i= 6 b= 1.3307080112533072 	step= 0.000531441
Q= -291.12316764706765
i= 7 b= 1.1220842079753766 	step= 0.0004782969
Q= -290.00108343909227
i= 8 b= 0.953591116028349 	step= 0.00043046721
Q= -289.0474923230639
i= 9 b= 0.8158529823148228 	step= 0.000387420489
Q= -288.2316393407491
i= 10 b= 0.7020695898260101 	step= 0.0003486784401
Q= -287.5295697509231
i= 11 b= 0.6072102710187437 	step= 0.00031381059609000004
Q= -286.92235947990434
i= 12 b= 0.5274885685215054 	step= 0.00028242953648100003
Q= -286.39487091138284
i= 13 b= 0.46001067300420573 	step= 0.00025418658283290005
Q

KeyboardInterrupt: 

In [ ]:
'''
np.save('hbeta.npy',hbeta)
np.save('Betat.npy',Betat)
np.save('Y.npy',Y)
np.save('X.npy',X)
'''


In [2]:
hbeta=np.load('hbeta.npy')
Betat=np.load('Betat.npy')
Y=np.load('Y.npy')
X=np.load('X.npy')

In [9]:
Lih(X[0],Y[0],hbeta[0])

array([[0.81187864, 0.18812136],
       [0.74203529, 0.25796471],
       [0.92807075, 0.07192925],
       [0.80034579, 0.19965421],
       [0.93856859, 0.06143141],
       [0.8302432 , 0.1697568 ],
       [0.94741357, 0.05258643],
       [0.8717806 , 0.1282194 ],
       [0.82508072, 0.17491928],
       [0.97069444, 0.02930556],
       [0.93005276, 0.06994724],
       [0.83764491, 0.16235509],
       [0.9179101 , 0.0820899 ],
       [0.94273119, 0.05726881],
       [0.73782066, 0.26217934],
       [0.767501  , 0.232499  ],
       [0.83887226, 0.16112774],
       [0.89944095, 0.10055905],
       [0.88434589, 0.11565411],
       [0.        , 1.        ],
       [0.81226178, 0.18773822],
       [0.91823518, 0.08176482],
       [0.73861329, 0.26138671],
       [0.83184985, 0.16815015],
       [0.66197626, 0.33802374],
       [0.96873593, 0.03126407],
       [0.9077245 , 0.0922755 ],
       [0.90234672, 0.09765328],
       [0.92037856, 0.07962144],
       [0.544789  , 0.455211  ],
       [0.

In [15]:
A=Lih(X[0],Y[0],hbeta[0])
B=A[:,0]-A[:,1]
c=np.array(np.where(B[:]<=0))
d=np.array(np.where(Y[0][:]==1))

In [23]:
c

array([[ 19,  40,  44,  66,  85,  86,  92, 109, 120, 181, 197, 260, 264,
        269, 286, 320, 380, 382, 392, 445, 451, 456, 483]], dtype=int64)

In [ ]:
Sc=np.zeros((M,N))
for i in range(M):
    A=Lih(X[i],Y[i],hbeta[i])
    B=A[:,0]-A[:,1]
    c=np.array(np.where(B[:]<=0)).shape

In [14]:
mse=hbeta-Betat

In [15]:
RMse=np.zeros(M)
for i in range(M):
    RMse[i]=np.sqrt(np.sum(mse[i]**2)/6)

In [16]:
RMse

array([0.11973127, 0.17021858, 0.27141978, 0.15446272, 0.11528667,
       0.12932834, 0.30002194, 0.2431961 , 0.17095843, 0.21777136,
       0.12428569, 0.19876331, 0.19192847, 0.14726149, 0.11704783,
       0.13657341, 0.21529787, 0.18429923, 0.13066423, 0.23540791,
       0.13810081, 0.16371621, 0.11205759, 0.21774761, 0.1412154 ,
       0.23364748, 0.15882849, 0.18650662, 0.20226183, 0.26298016,
       0.20939144, 0.10218752, 0.18485977, 0.14470734, 0.12103591,
       0.13604428, 0.19512713, 0.12038335, 0.11062807, 0.21177654,
       0.1339217 , 0.12691977, 0.23302819, 0.22834306, 0.13969978,
       0.27242373, 0.18654229, 0.15954002, 0.17315088, 0.10490443])

In [18]:
np.savetxt('Rmse.csv',RMse)

In [21]:
np.std(RMse)

0.049926490683483316

In [22]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [23]:
ss = StandardScaler()

In [26]:
xt = ss.fit_transform(X[0])


In [28]:
lr = LogisticRegression(C=1.0, tol=0.01)

In [32]:
lr.fit(xt,Y[0]).get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.01,
 'verbose': 0,
 'warm_start': False}

In [31]:
lr.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.01,
 'verbose': 0,
 'warm_start': False}